In [55]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [56]:
# load the trained model,scalar, onehotencoded pkl
model=load_model('model.h5')

# load the encoder and scalar
with open ('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open ('onehot_encoder.pkl','rb') as file:
    onehot_encoder=pickle.load(file)

with open ('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [57]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [58]:
one_encoded = onehot_encoder.transform([[input_data['Geography']]])
encoded_df=pd.DataFrame(one_encoded.toarray(),columns=onehot_encoder.get_feature_names_out(['Geography']))
encoded_df

/Users/priyanka/Desktop/NLP/EndToEndANN/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [59]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [60]:
# encode categorical label gender

input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [61]:
input_df=pd.concat([input_df.drop("Geography",axis=1),encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [62]:
input_scaled=scaler.transform(input_df)

In [63]:
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [64]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 79ms/step


array([[0.04507301]], dtype=float32)

In [67]:
prediction_proba=prediction[0][0]


In [68]:
prediction_proba

0.045073006

In [69]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
